In [2]:
###
###

In [3]:
%matplotlib inline
import pykat
import pylab as pl
import pandas
import matplotlib.pyplot as plt
import matplotlib
import csv
import math
import numpy as np
from pykat.parallel import parakat
from PropBeam import *

pandas.set_eng_float_format(accuracy=3, use_eng_prefix=True)

                                              ..-
    PyKat 1.0.9           _                  '(
                          \`.|\.__...-""""-_." )
       ..+-----.._        /  ' `            .-'
   . '            `:      7/* _/._\    \   (
  (        '::;;+;;:      `-"' =" /,`"" `) /
  L.        \`:::a:f            c_/     n_'
  ..`--...___`.  .    ,  
   `^-....____:   +.      www.gwoptics.org/pykat



In [4]:
####Perfect Mode-Matching Version:
katfile = "aLIGO_IFO_AWC_tuning_DCoff_maxtemsearch.kat"

# Add cavity parameter detectors
block1 ="""
tf sus 1 0 p $mech_fres $mech_Q
const mech_fres 1  # Approx. resonance frequency
const mech_Q    1M # Guess for suspension Q factor

# Differentially modulate the strain in the arms
fsig darm  LXarm 1 0 1
fsig darm2 LYarm 1 180 1

cp cavOMC x q

cp cavOMC x z

qnoisedS NSR_with_RP    1 $fs nOMC_AROC_trans

xaxis darm f log 5 5k 50

yaxis lin re:im
"""

block2 = """
cav cavIMC MC2 nMC2in MC2 nMC2refl
cav cavPRX PRMHR nPRMHRb ITMXHR nITMXs2
cav cavPRY PRMHR nPRMHRb ITMYHR nITMYs2
cav cavSRX SRMHR nSRMHRa ITMXHR nITMXs2
cav cavSRY SRMHR nSRMHRa ITMYHR nITMYs2
cav cavXARM ITMXHR nITMX2 ETMXHR nETMX1
cav cavYARM ITMYHR nITMY2 ETMYHR nETMY1
cav cavFC IMFC nIMFC3 IMFC nIMFC4
cav cavOMC OMC_HROC nOMC_HROC_refl OMC_HROC nOMC_HROC_in
"""

In [5]:
darm1 = []
maxtem = 6
for i in range(0,maxtem,1):
    input_line = "maxtem " + str(i)
    basekat1 = pykat.finesse.kat()
    basekat1.loadKatFile(katfile)
    basekat1.parseCommands(block1)
    basekat1.parseCommands(block2)
    basekat1.parseCommands(input_line)
    basekat1.undumpNodes() #undump the vacuum nodes
    kat1=basekat1.deepcopy() #copy kat code
    kat1.verbose = False
    ITMTL0 = kat1.ITMYTL.f.value
    diopters = 20e-6 #Estimated Avg fluctuations in TCS
    f_new = (1/ITMTL0 - diopters)**(-1)
    kat1.ITMYTL.f = f_new
    out1 = kat1.run()
    darm1.append(np.real(out1["NSR_with_RP"]))

In [6]:
block3 = """
cav cavIMC MC2 nMC2in MC2 nMC2refl
cav cavPRX PRMHR nPRMHRb ITMXHR nITMXs2
cav cavPRY PRMHR nPRMHRb ITMYHR nITMYs2
cav cavSRY SRMHR nSRMHRa ITMYHR nITMYs2
cav cavSRX SRMHR nSRMHRa ITMXHR nITMXs2
cav cavXARM ITMXHR nITMX2 ETMXHR nETMX1
cav cavYARM ITMYHR nITMY2 ETMYHR nETMY1
cav cavFC IMFC nIMFC3 IMFC nIMFC4
cav cavOMC OMC_HROC nOMC_HROC_refl OMC_HROC nOMC_HROC_in
"""

In [7]:
darm2 = []
for i in range(0,maxtem,1):
    input_line = "maxtem " + str(i)
    basekat2 = pykat.finesse.kat()
    basekat2.loadKatFile(katfile)
    basekat2.parseCommands(block1)
    basekat2.parseCommands(block3)
    basekat2.parseCommands(input_line)
    basekat2.undumpNodes() #undump the vacuum nodes
    kat2=basekat2.deepcopy() #copy kat code
    kat2.verbose = False
    ITMTL0 = kat2.ITMYTL.f.value
    diopters = 20e-6 #Estimated Avg fluctuations in TCS
    f_new = (1/ITMTL0 - diopters)**(-1)
    kat2.ITMYTL.f = f_new
    out2 = kat2.run()
    darm2.append(np.real(out2["NSR_with_RP"]))

In [8]:
maxtem_list

NameError: name 'maxtem_list' is not defined

In [ ]:
ratio = []
maxtem_list = []


for i in range(0,maxtem,1):
    maxtem_list.append(float(i))
    nratio = np.divide(darm1[i],darm2[i])
    ratio.append(nratio)

In [ ]:
matplotlib.RcParams.update({'font.size', 'Large'})
pl.figure(figsize=(19, 40), dpi=80)

for i in range(1,maxtem,1):
    sub = float("5"+"1"+str(i))
    pl.subplot(sub)
    pl.loglog(out1.x, darm1[i],out1.x, darm2[i],linewidth=3)
    pl.xlim(out1.x.min(),out1.x.max())
    pl.ylabel('h(t) for maxtem='+str(i), fontsize = 20)
    pl.grid(True, zorder=-1)
    pl.legend()

plt.xlabel('Freq', fontsize = 20)
    